In [3]:
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from matplotlib import pyplot as plt
import scipy.stats as sts
import scipy. integrate as intr
import pandas_datareader.data as web
import datetime
from scipy import optimize as opt
max_line_width=np.inf
import pprint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
start_date = datetime.datetime(1970, 10, 1)
end_date = datetime.datetime(2018, 7, 1)

In [6]:
RPCE = web.DataReader('PCECC96', 'fred', start_date, end_date)# Quarterly and Real
N_WIL = web.DataReader('WILL5000INDFC', 'fred', start_date, end_date)# Weekdays and Nominal
GDPDEF= web.DataReader('GDPDEF', 'fred', start_date, end_date)# Quarterly 
N_Rf = web.DataReader('TB3MS', 'fred', start_date, end_date) # Nominal and monthly
RPCE.tail(5)


,PCECC96
DATE,
2017-04-01,12515.860
2017-07-01,12584.906
2017-10-01,12706.369
2018-01-01,12722.839
2018-04-01,12847.820


In [7]:
N_Rf.tail(10)

,TB3MS
DATE,
2017-09-01,1.03
2017-10-01,1.07
2017-11-01,1.23
2017-12-01,1.32
2018-01-01,1.41
2018-02-01,1.57
2018-03-01,1.70
2018-04-01,1.76
2018-05-01,1.86


In [8]:
N_WIL.tail(10)

,WILL5000INDFC
DATE,
2018-06-18,130.88
2018-06-19,130.40
2018-06-20,130.75
2018-06-21,129.83
2018-06-22,130.02
2018-06-25,128.13
2018-06-26,128.50
2018-06-27,127.23
2018-06-28,127.99


In [9]:
N_Rf = N_Rf .fillna(method="ffill")
N_WIL= N_WIL.fillna(method="ffill")
N_Rf = N_Rf.resample('QS').first()
N_WIL = N_WIL.resample('QS').first()


lucas = pd.DataFrame(columns=['C', 'R', 'Rf', 'deflator'], index = RPCE.index)
lucas['C'] = RPCE
lucas['R'] = N_WIL
lucas['Rf'] = N_Rf / 100 + 1
lucas['deflator'] = (GDPDEF / 100)
lucas['deflator'] /= lucas['deflator'][0]
lucas['R'] = lucas['R'] / (lucas['deflator'])
# lucas['Rf'] = lucas['Rf'] / (lucas['deflator'])
lucas['R'] = lucas['R'] / lucas['R'].shift(1)

lucas.dropna(axis=0, inplace=True)
lucas.sample(10)

,C,R,Rf,deflator
DATE,,,,
1998-10-01,7983.316,0.839790,1.0396,3.429498
1975-04-01,3598.886,1.228650,1.0561,1.340316
1973-04-01,3565.349,0.906932,1.0626,1.123821
1989-10-01,5948.000,1.095945,1.0764,2.814835
1993-04-01,6426.703,1.032610,1.0287,3.115660
2003-01-01,9215.501,1.036894,1.0117,3.718943
2005-07-01,10145.297,1.025183,1.0322,3.980005
1992-01-01,6161.398,1.079728,1.0380,3.025889
2008-10-01,10483.376,0.902679,1.0067,4.306583


In [10]:
def objective_1(γ, *args):
    lucas, β = args
    R_1 = lucas['R']
    MU = lucas['C'] ** (-γ)
    m_1 = β * (MU / MU.shift(1))
    return 1 - m_1.cov(R_1) - m_1.mean() * R_1.mean()

In [11]:
β = 0.99
result_1 = opt.root(objective_1, 3.0,method = 'broyden1', args=(lucas, β))
result_1

     fun: array([5.08961637e-06])
 message: 'A solution was found at the specified tolerance.'
     nit: 4
  status: 1
 success: True
       x: array(1.35556038)

#6.1.2
$\gamma =0 $ represents a risk neutral agent. Whereas $\gamma = 1$ represents a log utility function where substitution and income effect just cancel each other,but it still shows risk aversion. Higher values than that show higher risk avers